In [1]:
import pandas

In [7]:
train_ids=pandas.read_pickle('data/train_biz_ids.pkl')
train_ids['bucket']='train'
val_ids=pandas.read_pickle('data/val_biz_ids.pkl')
val_ids['bucket']='val'
test_ids=pandas.read_pickle('data/test_biz_ids.pkl')
test_ids['bucket']='test'

In [6]:
print(train_ids.columns)

Index(['business_id', 'name', 'attributes.RestaurantsPriceRange2',
       'review_count', 'bucket'],
      dtype='object')


In [4]:
reviews_all=pandas.read_pickle('data/reviews_prepped.pkl')

In [8]:
bizdf=pandas.concat([train_ids,val_ids,test_ids],axis=0)

In [10]:
print(bizdf.shape)

(108082, 5)


In [9]:
print(reviews_all.columns)

Index(['review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny',
       'cool', 'date', 'name', 'attributes.RestaurantsPriceRange2',
       'review_count', 'date2', 'text_prep'],
      dtype='object')


In [19]:
reviews_all_with_bucket=pandas.merge(reviews_all,bizdf,on='business_id',how='outer')

In [20]:
print(reviews_all.shape)
print(reviews_all_with_bucket.shape)

(3257815, 13)
(3257818, 17)


In [13]:
print(reviews_all_with_bucket.columns)

Index(['review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny',
       'cool', 'date', 'name_x', 'attributes.RestaurantsPriceRange2_x',
       'review_count_x', 'date2', 'text_prep', 'name_y',
       'attributes.RestaurantsPriceRange2_y', 'review_count_y', 'bucket'],
      dtype='object')


In [21]:
print(reviews_all_with_bucket['name_x'].isna().sum())
print(reviews_all_with_bucket['name_y'].isna().sum())
#this means there are 3 businesses for which there are no reviews found

3
0


In [22]:
nomatch=reviews_all_with_bucket[reviews_all_with_bucket['name_x'].isna()]
reviews_all_with_bucket=reviews_all_with_bucket[~reviews_all_with_bucket['name_x'].isna()]

In [18]:
print(reviews_all_with_bucket.shape)

(3257815, 17)


In [23]:
print(nomatch['bucket'])

3257815    train
3257816    train
3257817    train
Name: bucket, dtype: object


In [24]:
#verify join is correct
chk=reviews_all_with_bucket.loc[~(reviews_all_with_bucket['name_x']==reviews_all_with_bucket['name_y'])]
print(chk.shape)

(0, 17)


In [28]:
reviews_all_with_bucket=reviews_all_with_bucket.drop(['name_y','attributes.RestaurantsPriceRange2_y','review_count_y'],axis=1)
reviews_all_with_bucket=reviews_all_with_bucket.rename(columns={
    'name_x':'name', 'attributes.RestaurantsPriceRange2_x':'pricerange','review_count_x':'review_count'})
print(reviews_all_with_bucket.columns)

Index(['review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny',
       'cool', 'date', 'name', 'pricerange', 'review_count', 'date2',
       'text_prep', 'bucket'],
      dtype='object')


In [29]:
train_reviews=reviews_all_with_bucket[reviews_all_with_bucket['bucket']=='train']
val_reviews=reviews_all_with_bucket[reviews_all_with_bucket['bucket']=='val']
test_reviews=reviews_all_with_bucket[reviews_all_with_bucket['bucket']=='test']

In [26]:
print(train_reviews.shape)
print(val_reviews.shape)
print(test_reviews.shape)

(2608302, 17)
(324584, 17)
(324929, 17)


In [30]:
train_reviews=train_reviews.drop(['bucket'],axis=1)
val_reviews=val_reviews.drop(['bucket'],axis=1)
test_reviews=test_reviews.drop(['bucket'],axis=1)

In [31]:
print(train_reviews.columns)

Index(['review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny',
       'cool', 'date', 'name', 'pricerange', 'review_count', 'date2',
       'text_prep'],
      dtype='object')


In [32]:
train_reviews.to_pickle("data/reviews_train.pkl")
val_reviews.to_pickle("data/reviews_val.pkl")
test_reviews.to_pickle("data/reviews_test.pkl")